In [ ]:
import os
import sys
import time
import pickle
import random
import numpy as np 
import pandas as pd
import json
from collections import OrderedDict
from sklearn.model_selection import KFold

global_start_t = time.time()
print('ok')

In [ ]:
!pip install rouge
!cp -r ../input/mrc-baidu-pytorch-baseline-ga/pytorch_baseline_ga/*  ./
!cp -r ../input/baidu-mrc-2021-competition-data/data  ./
!mkdir ./log

print('ok')

In [ ]:
def detect_train_dev_json():    
    print('in detect_train_dev_json()')
    
    json_file_path = '../input/baidu-mrc-2021-competition-data/data/train.json'
    train_fileJson = json.load(open(json_file_path))
    json_file_path = '../input/baidu-mrc-2021-competition-data/data/dev.json'
    dev_fileJson = json.load(open(json_file_path))
    train_fileJson['data'][0]['paragraphs'] += dev_fileJson['data'][0]['paragraphs']
    train_fileJson['data'][0]['paragraphs'] += dev_fileJson['data'][1]['paragraphs']
    print('len of merged data: ', len(train_fileJson['data'][0]['paragraphs']))
    
    paragraphs_lst = train_fileJson['data'][0]['paragraphs']
    ids_lst = []
    for para in paragraphs_lst:
        qid = para['qas'][0]['id']
        if qid in ids_lst:
            print('found duplicate qid is', qid)
        ids_lst.append(qid)

    print('merge_train_dev_json finished!')
    
detect_train_dev_json()

In [ ]:
import json
import os
import time
import pickle
import numpy as np

import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizerFast

from dataset.dataset import MrcDataset
from dataset.dataset import collate_fn
from models.model import MRC_model, MRC_model_tsg
from metric.metric4merge import compute_prediction_checklist_for_merge
from run import evaluate
from utils.finetuning_argparse import get_argparse
from utils.utils import seed_everything, init_logger, logger

from utils.evaluate_baidu import evaluate as evaluate_baidu
from utils.evaluate_baidu import read_mrc_dataset, read_model_prediction

def write_to_json(json_data, file_name):
    # 写入data文件
    objects = json.dumps(json_data, indent=4, ensure_ascii=False)
    with open(file_name, 'w', encoding='utf-8') as f:
        f.write(objects)

def softmax_func(x, return_list=False):
    if type(x)==list:
        x = np.array(x)
    exp_x = np.exp(x)
    x = exp_x / np.sum(exp_x)
    if return_list:
        return list(x)
    else:
        return x
    
def merge_dev_outcome(args):
    print('in merge_dev_outcome()')
    if not os.path.exists(args.output_dir):
        os.makedirs(args.output_dir)
        
    dev_json_files = [
        '../input/mrc-baidu-baseline-roberta-fold4-cv0-2027/roberta_large_output/dev_cv0_predictions_best.json',
        '../input/mrc-baidu-baseline-roberta-fold4-cv1-2027/roberta_large_output/dev_cv1_predictions_best.json',
        '../input/mrc-baidu-baseline-roberta-fold4-cv2-2027/roberta_large_output/dev_cv2_predictions_best.json',
        '../input/mrc-baidu-baseline-roberta-fold4-cv3-2027/roberta_large_output/dev_cv3_predictions_best.json',
    ]
    dev_outcome_merged = {}
    for dev_json_file in dev_json_files:
        dev_fileJson = json.load(open(dev_json_file))
        dev_outcome_merged.update(dev_fileJson)
    print('len of dev_outcome_merged is ', len(dev_outcome_merged))
    
    dev_outcome_merged_file = os.path.join(args.output_dir, 'dev_predictions_merged.json')
    with open(dev_outcome_merged_file, "w", encoding='utf-8') as writer:
        writer.write(json.dumps(dev_outcome_merged, ensure_ascii=False, indent=4) + "\n")
    
    ref_ans = read_mrc_dataset(os.path.join(args.output_dir, 'tain_dev_merged.json'))
    pred_ans = read_model_prediction(dev_outcome_merged_file)
    #print('before F1, EM, TOTAL, SKIP = base_evaluate(ref_ans, pred_ans, False)')
    baidu_F1, baidu_EM, TOTAL, SKIP = evaluate_baidu(ref_ans, pred_ans, False)
    
    print(f'baidu_F1: {baidu_F1:.5f}, baidu_EM: {baidu_EM:.5f}')

def merge_train_dev_json(args):    
    print('in merge_train_dev_json()')
    
    if not os.path.exists(args.output_dir):
        os.makedirs(args.output_dir)
        
    json_file_path = '../input/baidu-mrc-2021-competition-data/data/train.json'
    train_fileJson = json.load(open(json_file_path))
    json_file_path = '../input/baidu-mrc-2021-competition-data/data/dev.json'
    dev_fileJson = json.load(open(json_file_path))
    train_fileJson['data'][0]['paragraphs'] += dev_fileJson['data'][0]['paragraphs']
    train_fileJson['data'][0]['paragraphs'] += dev_fileJson['data'][1]['paragraphs']
    print('len of merged data: ', len(train_fileJson['data'][0]['paragraphs']))
    
    write_to_json(train_fileJson, os.path.join(args.output_dir, 'tain_dev_merged.json'))
    print('merge_train_dev_json finished!')
    
def merge_outcome(data_loader, args, prefix=""):
    print('from tusonggao, in merge_outcome()')

    all_start_logits = []
    all_end_logits = []
    all_cls_logits = []

    pkl_files = [
                 '../input/mrc-baidu-baseline-roberta-fold4-cv0-2027/roberta_large_output/test1_predictions_detail.pkl',
                 '../input/mrc-baidu-baseline-roberta-fold4-cv1-2027/roberta_large_output/test1_predictions_detail.pkl',
                 '../input/mrc-baidu-baseline-roberta-fold4-cv2-2027/roberta_large_output/test1_predictions_detail.pkl',
                 '../input/mrc-baidu-baseline-roberta-fold4-cv3-2027/roberta_large_output/test1_predictions_detail.pkl',
    ]
    print('len of pkl_files is ', len(pkl_files))
    
    if not os.path.exists(args.output_dir):
        os.makedirs(args.output_dir)

    print('from tusonggao, starts now!')
    all_start_logits, all_end_logits, all_cls_logits = [], [], []
    for pkl_file in pkl_files:
        dump_detial = pickle.load(open(pkl_file, "rb"))
        all_start_logits_part = dump_detial['all_start_logits']
        all_end_logits_part = dump_detial['all_end_logits']
        all_cls_logits_part = dump_detial['all_cls_logits']
        print('for pkl_file: ', pkl_file, 'len of all_start_logits_part: ', len(all_start_logits_part))
        assert len(all_start_logits_part)==len(all_end_logits_part)==len(all_cls_logits_part), 'should all be the same length'
        all_start_logits_part = [softmax_func(all_start_logits_part[i]) for i in range(len(all_start_logits_part))]
        all_end_logits_part = [softmax_func(all_end_logits_part[i]) for i in range(len(all_end_logits_part))]
        all_cls_logits_part = [softmax_func(all_cls_logits_part[i]) for i in range(len(all_cls_logits_part))]

        if all_start_logits==[]:
            all_start_logits = all_start_logits_part
            all_end_logits = all_end_logits_part
            all_cls_logits = all_cls_logits_part
        else:
            for i in range(len(all_start_logits)):
                assert all_start_logits[i].shape==all_start_logits_part[i].shape, 'shape should be the same'
                all_start_logits[i] = all_start_logits[i] + all_start_logits_part[i]
                all_end_logits[i] = all_end_logits[i] + all_end_logits_part[i]
                all_cls_logits[i] = all_cls_logits[i] + all_cls_logits_part[i]

    for i in range(len(all_start_logits)):
        all_start_logits[i] = all_start_logits[i] / len(pkl_files)
        all_end_logits[i] = all_end_logits[i] / len(pkl_files)
        all_cls_logits[i] = all_cls_logits[i] / len(pkl_files)
        
    print('from tusonggao, ends now!')
    
    merged_predictions_detail = {'all_start_logits': all_start_logits,
                                 'all_end_logits': all_end_logits,
                                 'all_cls_logits': all_cls_logits}
    merged_detail_dump_filename = os.path.join(args.output_dir, prefix+'_predictions_detail_merged.pkl')
    pickle.dump(merged_predictions_detail, open(merged_detail_dump_filename, 'wb'), protocol=4)
    print('dump merged_predictions_detail file succeed now!')

    print('from tusonggao len of all_start_logits: ', len(all_start_logits))
    print('from tusonggao len of all_end_logits: ', len(all_end_logits))
    print('from tusonggao len of all_cls_logits: ', len(all_cls_logits))
    assert len(all_start_logits)==len(all_end_logits)==len(all_cls_logits), 'should all be the same length 222'

    print('from tusonggao args.max_answer_length is ', args.max_answer_length) 
    all_predictions, all_nbest_json, all_cls_predictions = compute_prediction_checklist_for_merge(
        data_loader.dataset.examples, 
        data_loader.dataset.tokenized_examples,
        (all_start_logits, all_end_logits,all_cls_logits), 
        True, 
        args.n_best_size,
        args.max_answer_length, 
        args.cls_threshold
    )

    print('from tusonggao dump merged file now')
    with open(os.path.join(args.output_dir,prefix+'_predictions_merged.json'), "w", encoding='utf-8') as writer:
        writer.write(
            json.dumps(
                all_predictions, ensure_ascii=False, indent=4) + "\n")
    print('from tusonggao dump merged file succeed now!')

def main(args):
    print('in main()')
    os.environ["TOKENIZERS_PARALLELISM"] = "false"
    seed_everything(args.seed)

    print('from tusonggao, args.cls_threshold: ', args.cls_threshold)
    print('from tusonggao, args.cls_threshold: ', args.test_file_name)

    # tokenizer
    tokenizer = BertTokenizerFast.from_pretrained(args.model_name_or_path)

    print('from tusonggao got test_iter start now!')
    # Dataset & Dataloader
    test_dataset = MrcDataset(args,
                              json_path=args.test_file_name,
                              tokenizer=tokenizer)
    test_iter = DataLoader(test_dataset,
                           shuffle=False,
                           batch_size=args.per_gpu_eval_batch_size,
                           collate_fn=collate_fn,
                           num_workers=24)
    print('from tusonggao got test_iter succeed now!')

    logger.info("The nums of the test_dataset examples is {}".format(len(test_dataset.examples)))
    logger.info("The nums of the test_dataset features is {}".format(len(test_dataset)))

    merge_outcome(test_iter, args, prefix="test")

class CFG:
    cls_threshold = 0.6
    test_file_name = './data/test1.json'
    model_name_or_path = 'hfl/chinese-roberta-wwm-ext-large'
    per_gpu_eval_batch_size = 2
    output_dir = './merged_outcome'
    max_len = 512
    max_answer_length = 150
    seed = 66
    n_best_size = 5
    stride = 100
    
if __name__ == "__main__":
    args = CFG()
    
    merge_train_dev_json(args)
    merge_dev_outcome(args)
    main(args)
    print('finished now!')

In [ ]:
!ls ./merged_outcome

In [ ]:
print('\nfinished now, total cost time: ', time.time() - global_start_t)